# Data Generation

The data used in the Learning Management Database was generated using Mock data from various sources for tables like students, instructors, and courses, while other data was generated using patterns by joining the mock tables with some patterns, for example: the topics table is generated by adding chapters 1 to 4 for every course. the following queries are used to generate such patterns

### Clean Database Table From Test Data
```sql

-- 4 remove dummy data from stage 4 tables
DELETE FROM Programs.Answers

-- 3 remove dummy data from stage 3 tables
DELETE FROM Programs.Questions

-- 2 remove dummy data from stage 2 tables
DELETE FROM Programs.Exams
DELETE FROM Programs.Topics;
DELETE FROM Programs.ProgramCourses;
DELETE FROM Institutions.ClassRooms;

-- 1 remove dummy data for stage 1 tables
DELETE FROM Institutions.TimeSlots;
DELETE FROM Institutions.Semesters;
DELETE FROM Institutions.Departments;
DELETE FROM Programs.Courses;
DELETE FROM Programs.Assessments;
DELETE FROM Institutions.Programs;
DELETE FROM Institutions.institutions;

-- 4. reset identity of stage 4 tables
DBCC CHECKIDENT ('Programs.Answers', RESEED, 0);

-- 3. reset identity of stage 3 tables
DBCC CHECKIDENT ('Programs.Questions', RESEED, 0);

-- 2. reset identity of stage 2 tables
DBCC CHECKIDENT ('Programs.Exams', RESEED, 0);
DBCC CHECKIDENT ('Programs.Topics', RESEED, 0);
-- DBCC CHECKIDENT ('Programs.ProgramCourses', RESEED, 0); has no identity column
DBCC CHECKIDENT ('Institutions.ClassRooms', RESEED, 0);

-- 1. reset identity of stage 1 tables
DBCC CHECKIDENT ('Institutions.Departments', RESEED, 0);
DBCC CHECKIDENT ('Institutions.TimeSlots', RESEED, 0);
DBCC CHECKIDENT ('Institutions.Semesters', RESEED, 0);
DBCC CHECKIDENT ('Programs.Courses', RESEED, 0);
DBCC CHECKIDENT ('Programs.Assessments', RESEED, 0);
DBCC CHECKIDENT ('Institutions.Programs', RESEED, 0);
DBCC CHECKIDENT ('Institutions.institutions', RESEED, 0);
```

### Generate Exams Table Data
```sql
with exams as (
select 'mid-term' as exam_name,1 as exam_order
union
select 'pre-final',2
union
select 'final', 3
)
select ROW_NUMBER() OVER(ORDER BY course_id) as exam_id, *
from Programs.Courses, exams
where 
	(course_id <= 6) 
	or (course_id between 8 and 25)
	or (course_id > 25 and exam_order = 3)
order by course_id, exam_order
;
```

### Generate Questions Table Data

```sql
with questions_ as (
select 'question 1' as question_name,1 as questions_order
union
select 'question 2' as question_name,2 as questions_order
union
select 'question 3' as question_name,3 as questions_order
union
select 'question 4' as question_name,4 as questions_order
union
select 'question 5' as question_name,5 as questions_order
union
select 'question 6' as question_name,6 as questions_order
union
select 'question 7' as question_name,7 as questions_order
union
select 'question 8' as question_name,8 as questions_order
),
questions2 as(
select *
from questions_
),
course_topic as (
select exam_id, t.topic_id, c.course_id, e.exam_creation_date, e.exam_name
from Programs.Topics t 
	join programs.Courses c on c.course_id= t.course_id
	join Programs.Exams e on e.course_id =c.course_id
),
cte as (
select *, DENSE_RANK() over(partition by course_id order by topic_id) as question_filter 
from course_topic c, questions2
),
questions_table as (
select  question_name as question_text, exam_id, topic_id 
from cte  
where  
(exam_name in ('mid-term') and questions_order <= 5 and questions_order = question_filter and question_name in ('question 1' , 'question 2' )) 
or (exam_name in ( 'pre-final') and questions_order <= 5 and questions_order = question_filter and question_name in ('question 3' , 'question 4' )) 
or (exam_name = 'final' and questions_order <= 8 and (questions_order-4 = question_filter or questions_order = question_filter))
)
select ROW_NUMBER() OVER (ORDER BY exam_id, question_text) as question_id,* 
from questions_table
order by  exam_id, question_text
```

### Generate answers table

```sql
with answers_ as (
select 'Answer A' as answer_name,1 as answer_order
union
select 'Answer B' as answer_name,2 as answer_order
union
select 'Answer C' as answer_name,3 as answer_order
union
select 'Answer D' as answer_name,4 as answer_order
union
select 'Answer E' as answer_name,5 as answer_order
),
questions as(
select * from Programs.Questions
),
answers2 as (
select row_number() over(order by question_id, answer_order)as answer_id, answer_name as answer_text,question_id, answer_order
from questions, answers_
)
select answer_id, answer_text, question_id, 
case 
	when answer_text = 'Answer D' then 1
	else 0
end as right_answer
from answers2
order by question_id, answer_order
```

### Generate Class Schedule Table

#### Generate Schedule and courses Helper Tables
output is copied from the SQl DB Client to Intermediate staging files
```sql
SELECT * 
FROM 
	Institutions.ClassRooms, Institutions.TimeSlots
WHERE 
	day_of_week NOT IN('Saturday','Friday') 
	AND time_slot_start_time < '15:30:00.0000000'
	AND (classroom_name LIKE '%A'
		OR classroom_name LIKE'%B'
		OR classroom_name LIKE '%C'
		OR classroom_name LIKE '%D'
		)
ORDER BY institution_id, classroom_name, time_slot_id, time_slot_start_time
;
```
```sql
SELECT 
	ip.institution_program_id,
	c.course_year_number,
	i.institution_id, 
	p.program_name,
	p.program_id, 
	c.course_name,
	c.course_id,
	pc.course_program_id, 
	s.semester_name
FROM Institutions.InstitutionPrograms ip 
JOIN Institutions.Institutions i ON ip.institution_id = i.institution_id
JOIN Institutions.Programs p ON ip.program_id = p.program_id
JOIN Programs.ProgramCourses pc ON pc.program_id = p.program_id 
JOIN Programs.Courses c ON c.course_id= pc.course_id
JOIN Programs.CourseSemester cs ON cs.course_id=c.course_id
JOIN Institutions.Semesters s ON s.semester_id = cs.semester_id
WHERE	semester_name IN ('Fall', 'October Intake', 'December Intake') 
	--and program_name not in ('Embedded Systems Track', 'Cyber Security Track')
ORDER BY  ip.institution_program_id,c.course_year_number,i.institution_id, p.program_name, c.course_name;
```

#### Generate program courses, classrooms, and time slots
use source data from the Intermediate staging data files to generate:
program course id, class room id, and time slot id and copy manually to the CSV source file Institutions_classSchedule  
```python
import pandas as pd

df_courses = pd.read_csv('./data_generation_helper_tables/courses.csv')
df_schedule= pd.read_csv('./data_generation_helper_tables/schedule.csv')

df_courses.rename(columns = {"institution_id": "course_institution_id"}, inplace=True)
df_schedule.rename(columns = {"institution_id": "schedule_institution_id"}, inplace=True)

df_schedule_courses= pd.DataFrame(columns= df_schedule.columns.append(df_courses.columns))
display(df_schedule_courses)

for institution_id in df_courses.course_institution_id.unique():
    print(f"---------- Institution {institution_id} ----------")
    
    df_schedule_filter= df_schedule[df_schedule["schedule_institution_id"] == institution_id]
    df_courses_filter = df_courses[df_courses["course_institution_id"] == institution_id]
    #display(df)

    for schedule_index, schedule_row in df_schedule_filter.iterrows():
        #print("Schedule:")
        #print(schedule_row[["classroom_name", "day_of_week", "time_slot_start_time"]])
        #print()
        for course_index, course_row in df_courses_filter.iterrows():
            #print("Course")
            #print(course_row[["course_name", "program_name", "institution_id"]])
            #print()
            if (df_schedule_courses[
                    (df_schedule_courses["course_name"] == course_row["course_name"])
                    & (df_schedule_courses["program_name"] == course_row["program_name"])
                    & (df_schedule_courses["course_institution_id"] == course_row["course_institution_id"])]
                .shape[0] < 5 
                ):
                
                new_row= pd.concat([schedule_row, course_row]).to_frame().transpose()
                df_schedule_courses = pd.concat([df_schedule_courses, new_row], ignore_index=True)
                # display(df_schedule_courses)
                # print(df_schedule_courses[
                #     (df_schedule_courses["course_name"] == course_row["course_name"])
                #     & (df_schedule_courses["program_name"] == course_row["program_name"])
                #     & (df_schedule_courses["course_institution_id"] == course_row["course_institution_id"])]
                # .shape[0])
                break
            else:
                continue
display(df_schedule_courses)
df_schedule_courses.to_csv("./data_generation_helper_tables/schedule_computed.csv")
```

#### Generate Instructors Data
the instructors_id where added manually into the source csv file Institutions_ClassSchedule
```sql
select * 
	from Institutions.InstitutionDepartments id 
	join Institutions.Institutions i on id.institution_id = i.institution_id
	join Institutions.Departments d on id.department_id = d.department_id
	join Institutions.Instructors ir on ir.instructor_institution_department_id = id.institution_department_id
where id.[institution_id]= 1;

select * 
	from Institutions.InstitutionDepartments id 
	join Institutions.Institutions i on id.institution_id = i.institution_id
	join Institutions.Departments d on id.department_id = d.department_id
	join Institutions.Instructors ir on ir.instructor_institution_department_id = id.institution_department_id
where id.[institution_id]= 2;

select * 
	from Institutions.InstitutionDepartments id 
	join Institutions.Institutions i on id.institution_id = i.institution_id
	join Institutions.Departments d on id.department_id = d.department_id
	join Institutions.Instructors ir on ir.instructor_institution_department_id = id.institution_department_id
where id.[institution_id]= 3;
```

### Generate Student Enrollments

```sql
USE LMS;

BEGIN TRANSACTION;

-- Drop temp tables if they exist
IF OBJECT_ID('tempdb..#TempScheduleVerbose') IS NOT NULL
    DROP TABLE #TempScheduleVerbose;

IF OBJECT_ID('tempdb..#TempStudentsVerbose') IS NOT NULL
    DROP TABLE #TempStudentsVerbose;

IF OBJECT_ID('tempdb..#TempStudentSchedule') IS NOT NULL
    DROP TABLE #TempStudentSchedule;

-- Create temp tables
-- Schedule with verbose names not only ids
SELECT
	cs.schedule_id, cs.program_course_id,
	p.program_name, p.program_id,
	c.course_name, c.course_year_number, c.course_id,
	cr.classroom_name, cs.classroom_max_size, cr.classroom_id,  
	ts.day_of_week, ts.time_slot_start_time, ts.time_slot_id,
	i.instructor_id,id.institution_id,id.department_id
INTO #TempScheduleVerbose
FROM institutions.ClassSchedule cs
JOIN Institutions.Instructors i ON i.instructor_id = cs.instructor_id
JOIN [Institutions].[InstitutionDepartments] id ON id.institution_department_id = i.instructor_institution_department_id
JOIN Institutions.TimeSlots ts ON ts.time_slot_id = cs.time_slot_id
JOIN Programs.ProgramCourses pg ON pg.program_course_id = cs.program_course_id
JOIN Programs.Courses c ON c.course_id = pg.course_id
JOin Institutions.Programs p ON p.program_id= pg.program_id
JOIN Institutions.ClassRooms cr ON cr.classroom_id = cs.classroom_id

--sudents with their enrolled verbose instituiton and program
SELECT 
	s.[student_year_number], s.student_institution_program_id, s.student_id, 
	i.institution_name,i.institution_id,  
	p.program_name, p.program_id  
INTO #TempStudentsVerbose
FROM Students.Students s
JOIN Institutions.InstitutionPrograms ip ON ip.institution_program_id = s.student_institution_program_id
JOIN Institutions.Institutions i ON i.institution_id = ip.institution_id
JOIN Institutions.Programs p on p.program_id = ip.program_id;

-- test temp tables
--SELECT * FROM #TempScheduleVerbose;
--SELECT * FROM #TempStudentsVerbose; 

-- temp table to populate 
CREATE TABLE #TempStudentSchedule(
	student_schedule_id INT IDENTITY(1,1) PRIMARY KEY,
	student_id INT,
	institution_id INT,
	program_id INT,
	course_id INT,
	schedule_id INT,
	enrollment_date Date NULL DEFAULT GETDATE()
);

-- SELECT * FROM #TempStudentSchedule;

--==============================================================================--
--==============================================================================--
TRUNCATE TABLE #TempStudentSchedule;
-- declare variables to hold SCHEDULE table row values
DECLARE 
	@schedule_id INT, 
	@program_course_id INT,
	@program_name VARCHAR(50),
	@program_id INT,
	@course_name VARCHAR(30), 
	@course_year_number INT, 
	@course_id INT,
	@classroom_name VARCHAR(5),
	@classroom_max_size INT,
	@classroom_id INT,
	@day_of_week VARCHAR(20),
	@time_slot_start_time TIME,
	@time_slot_id INT,
	@instructor_id INT,
	@institution_id INT,
	@department_id INT

-- declare variables to hold STUDENTS table row values  
DECLARE
	@student_year_number INT,
	@student_institution_program_id INT,
	@student_id INT,
	@student_institution_name VARCHAR(50),
	@student_institution_id INT,
	@student_program_name VARCHAR(30),
	@student_program_id INT
	

-- local fetch status to seprate inner loop's (studnets) fetch status
	-- from outer loops's (schedule) fetch status
DECLARE 
	@schedule_fetch_status INT,
	@student_fetch_status INT

-- Reset fetch status variables
SET @schedule_fetch_status = 0;
SET @student_fetch_status = 0;

DECLARE cursor_student CURSOR FOR 
	SELECT * FROM #TempStudentsVerbose;
DECLARE cursor_schedule CURSOR FOR 
   SELECT * FROM #TempScheduleVerbose;

-- Outer Loop (Schedule): iterate over classroom time slots to assign students until max capacity
OPEN cursor_schedule;
WHILE @schedule_fetch_status = 0 
BEGIN
    FETCH NEXT FROM cursor_schedule 
		INTO 
			@schedule_id, 
			@program_course_id, @program_name,@program_id,
			@course_name, @course_year_number, @course_id,
			@classroom_name, @classroom_max_size, @classroom_id,	
			@day_of_week, @time_slot_start_time, @time_slot_id,
			@instructor_id, @institution_id, @department_id;
	SET @schedule_fetch_status = @@FETCH_STATUS
	
	PRINT '--------------------------------------------------------'
	PRINT 'Schedule: ' + CAST(@schedule_id AS VARCHAR) + ' | Program: ' + CAST(@program_name AS VARCHAR)
			+ ' | Course Name: ' + CAST(@course_name AS VARCHAR) + ' | Course Year: '  + CAST(@course_year_number AS VARCHAR)
	PRINT '--------------------------------------------------------'
	
	DECLARE @total_students INT;
	DECLARE @student_in_class INT;
	DECLARE @student_course_timeslots_number INT;

	SET @total_students = (SELECT COUNT(*) FROM #TempStudentSchedule WHERE schedule_id = @schedule_id);
	SET @student_in_class= (SELECT COUNT(*) FROM #TempStudentSchedule WHERE schedule_id = @schedule_id AND student_id = @student_id);
	SET @student_course_timeslots_number = (SELECT COUNT(*) FROM #TempStudentSchedule WHERE student_id = @student_id AND course_id = @course_id AND program_id = @program_id)



	-- Inner Loop (Students): add students to the classroom time slot based on program, course, year, and institution conditions
	OPEN cursor_student;
	SET @student_fetch_status = 0;
	WHILE 
		@student_fetch_status = 0
		AND @total_students < @classroom_max_size
	BEGIN
		FETCH NEXT FROM cursor_student 
		INTO 	
			@student_year_number, 
			@student_institution_program_id, @student_id, 
			@student_institution_name, @student_institution_id, 
			@student_program_name,@student_program_id
		
		SET @student_fetch_status = @@FETCH_STATUS
		SET @student_in_class= (SELECT COUNT(*) FROM #TempStudentSchedule WHERE schedule_id = @schedule_id AND student_id = @student_id);
		SET @total_students = (SELECT COUNT(*) FROM #TempStudentSchedule WHERE schedule_id = @schedule_id);
		SET @student_course_timeslots_number = (SELECT COUNT(*) FROM #TempStudentSchedule WHERE student_id = @student_id AND course_id = @course_id AND program_id = @program_id)

		IF @student_fetch_status = 0
		BEGIN
			PRINT '0- Student_id: ' + CAST(@student_id AS VARCHAR)
			PRINT '1- Max Students: '+ CAST(@total_students AS VARCHAR) + '/' + CAST(@classroom_max_size AS VARCHAR);
			PRINT '2- Is Student in this Class TimeSlot?: '+ CAST( @student_in_class AS VARCHAR) +' (Student id: ' + CAST( @student_id AS VARCHAR)+')';
			PRINT '3- IS Student Enrolled in 6 Classes?: '+ CAST(@student_course_timeslots_number AS VARCHAR) + '/2';
			PRINT '4- Same Program? '+ ' Schedule Program: '+ CAST( @program_id AS VARCHAR) + ' Student Program: '+ CAST(@student_program_id AS VARCHAR) ;
			PRINT '5- Same Institution? '+ ' Schedule Institution: '+ CAST( @institution_id AS VARCHAR) + ' Student Institution: '+ CAST(@student_institution_id AS VARCHAR);
			PRINT '6- Same Year Number? '+ ' Schedule Year: '+ CAST( @course_year_number AS VARCHAR) + ' Student Year: '+ CAST(@student_year_number AS VARCHAR);
			PRINT ''
			-- PRINT 'FetchStatusSchedule: ' + CAST(@schedule_fetch_status AS VARCHAR)  
			--	'	FetchStatusStudent: ' + CAST(@student_fetch_status AS VARCHAR);
			IF
				@total_students < @classroom_max_size
				AND @student_in_class = 0
				AND @student_course_timeslots_number < 2
				AND @program_id = @student_program_id 
				AND @student_institution_id = @institution_id 
				AND @course_year_number = @student_year_number
			BEGIN
				PRINT 'ALL CONDITIONS MET'
				INSERT INTO #TempStudentSchedule(student_id,institution_id,program_id, course_id,schedule_id) 
					VALUES (@student_id,@institution_id,@program_id,@course_id,@schedule_id);
			END
		END
	END
	CLOSE cursor_student;

END;
CLOSE cursor_schedule;
DEALLOCATE cursor_student;
DEALLOCATE cursor_schedule;

COMMIT TRANSACTION;


SELECT 
	 s.student_id, 
	 i.institution_name, p.program_name, 
	 c.course_name, cr.classroom_name,
	 ts.day_of_week, ts.time_slot_start_time, ts.time_slot_id,
	 tss.schedule_id, course_year_number
FROM #TempStudentSchedule tss
JOIN Students.Students s ON s.student_id = tss.student_id
JOIN Institutions.Institutions i ON  i.institution_id = tss.institution_id
JOIN Institutions.Programs p ON p.program_id = tss.program_id
JOIN Programs.Courses c ON c.course_id = tss.course_id
JOIN Institutions.ClassSchedule cs ON cs.schedule_id = tss.schedule_id
JOIN Institutions.TimeSlots ts ON ts.time_slot_id = cs.time_slot_id
JOIN Institutions.ClassRooms cr ON cr.classroom_id = cs.classroom_id
order by 1
;


SELECT COUNT(DISTINCT student_id) FROM #TempStudentSchedule;

SELECT COUNT(*) first_year_student_course_classes FROM #TempStudentSchedule WHERE student_id =1 group by course_id;

SELECT GETDATE() AS enrollment_date_time, student_id, schedule_id FROM #TempStudentSchedule;
```

### Generate Students Course Results

#### generate the student_id, program course id and enrollment_date
```sql
select 
	student_id, program_course_id, se.enrollment_datetime
from Students.StudentEnrollments se
join [Institutions].[ClassSchedule] cs 
	on cs.schedule_id = se.schedule_id
 group by student_id, program_course_id , se.enrollment_datetime
order by student_id, program_course_id;
```

#### generate the course results in percentages and the course status
```python
import numpy as np
import pandas as pd

standard_of_deviation = 25
mean= 75
samples=4000

data = np.random.normal(mean, standard_of_deviation, samples)
percentage_data = data[(data >= 0) & (data <= 100)]

if len(percentage_data) >= 3120:
    # Transpose the array to convert it into a single-column DataFrame
    df = pd.DataFrame(percentage_data.reshape(-1, 1), columns=['course_result_percentage']).round(2)

    # Clip the DataFrame to a maximum length of 3120 rows
    df = df.iloc[:3120]
    
    # Set 10% of the values to NaN
    num_values_to_null = int(0.1 * len(df))
    random_indices = np.random.choice(len(df), num_values_to_null, replace=False)
    df.loc[random_indices, 'course_result_percentage'] = np.nan
    
    df["course_status"] = np.select(
        [
            df["course_result_percentage"].isnull(),
            df["course_result_percentage"] >= 65,
            df["course_result_percentage"] < 65
        ],
        [
            "incomplete",
            "Pass",
            "Fail"
        ],
        default='Error'
    )
    display(df[df["course_status"] == "Pass"].head(2))
    display(df[df["course_status"] == "Fail"].head(2))
    display(df[df["course_status"] == "incomplete"].head(2))
    
    df.to_csv("./data_generation_helper_tables/course_result_percentage_computed.csv")
else:
    print("Resutl is less than desired percentage column length")
    print(len(percentage_data))
```

### Generate Students Assessments Scores

```sql
with cte as (
select 
	*
from Students.CourseResults cr
), cte2 as (
select *, 
	case
	when assessment_id = 1 then 5
	when program_course_id <= 36 and assessment_name like 'quiz %' then 5
	when program_course_id <= 36 and assessment_name like 'lab %' then 5
	when program_course_id <= 36 and assessment_name like 'project %' then 5
	when program_course_id <= 36 and assessment_id = 19 then 40
	when program_course_id <= 36 and assessment_name like '% exam' then 15
	when program_course_id > 36 and assessment_name like 'project %' then 35
	when program_course_id > 36 and assessment_name like 'lab %' then 5
	when program_course_id > 36 and assessment_id = 19 then 15
	else NULL
	end as assessment_total_score
	from cte, Programs.Assessments
where 
	(program_course_id <= 3 and assessment_id in (1,2,3,7,8,12,17,18,19))
	or (program_course_id > 3 and assessment_id in (1,7,8,12,13,19))
), cte3 as(
select *,
		(course_result_percentage)/100 * assessment_total_score as student_score 
from cte2
)
select student_id, program_course_id, assessment_id, FORMAT(student_score, 'N2') AS student_score, assessment_total_score 
from cte3
order by student_id, program_course_id, assessment_id;
```

### Generate Exam Submissions

```sql
WITH correct_submissions AS(
SELECT 
	cr.student_id, cr.program_course_id, 
	pc.course_id, pc.program_id, 
	q.exam_id, q.question_id,q.topic_id, q.question_text,
	CASE 
		WHEN
		FLOOR((course_result_percentage/100) *
		COUNT(*) OVER(PARTITION BY cr.student_id, cr.program_course_id, e.exam_id))  >=
		ROW_NUMBER() OVER(PARTITION BY cr.student_id, cr.program_course_id, e.exam_id ORDER BY q.question_id)
			THEN 1
		ELSE 0
	END AS right_answer 
FROM [LMS].[Students].[CourseResults] cr
JOIN [LMS].[Students].[AssessmentScores] a 
	ON a.student_id = cr.student_id
	AND a.[program_course_id] = cr.program_course_id
JOIN [Programs].[ProgramCourses] pc
	ON pc.program_course_id = cr.program_course_id
JOIN [Programs].[Exams] e 
	ON e.course_id = pc.course_id and e.exam_name = 'final' -- 3120 final exams for the 600 students courses
JOIN [Programs].[Questions] q -- 3120 * 8 questions for the final exams
	ON q.exam_id = e.exam_id
WHERE assessment_id = 19)
SELECT GETDATE() AS exam_submission_datetime,  student_id, exam_id,ans.question_id, answer_id
FROM 
	correct_submissions cs
JOIN [Programs].[Answers] ans -- 3120 * 8 *  5 Answers
	ON ans.question_id = cs.question_id
	AND ans.right_answer = cs.right_answer
WHERE ans.answer_text NOT IN ('Answer A', 'Answer C', 'Answer E')
ORDER BY student_id, program_course_id, exam_id, cs.question_id, answer_id
```